In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path

# Configuration
SIM_RES = 2500
OUTPUT_DIR = Path('/mnt/home/mlee1/ceph/hydro_replace_fields')

# All snapshots with redshifts (from TNG docs)
SNAPSHOT_REDSHIFTS = {
    29: 2.32, 31: 2.14, 33: 1.97, 35: 1.82, 38: 1.63, 41: 1.47, 43: 1.36,
    46: 1.21, 49: 1.08, 52: 0.97, 56: 0.85, 59: 0.76, 63: 0.65, 67: 0.55,
    71: 0.46, 76: 0.35, 80: 0.27, 85: 0.18, 90: 0.10, 96: 0.02, 99: 0.0
}
SNAPSHOTS = sorted(SNAPSHOT_REDSHIFTS.keys())

# Mass bins for analysis
MASS_BIN_EDGES = [10**12.5, 10**13.0, 10**13.5, 10**14.0, 10**15.0]
MASS_BIN_LABELS = ['12.5-13.0', '13.0-13.5', '13.5-14.0', '>14.0']

# BCM models
BCM_MODELS = ['Arico20', 'Schneider19', 'Schneider25']

print("Profile Validation Notebook")
print(f"Simulation: L205n{SIM_RES}TNG")
print(f"Snapshots: {len(SNAPSHOTS)} ({SNAPSHOTS[0]}-{SNAPSHOTS[-1]})")
print(f"Redshift range: z={SNAPSHOT_REDSHIFTS[SNAPSHOTS[-1]]:.2f} to z={SNAPSHOT_REDSHIFTS[SNAPSHOTS[0]]:.2f}")

In [ ]:
# Check which profile files exist
# New format: profiles_spherical_snap*.h5, profiles_bcm_snap*.h5
# Old format: snap*/profiles.h5

print("Profile File Inventory:")
print("=" * 80)

profile_inventory = {}

for snap in SNAPSHOTS:
    sim_dir = OUTPUT_DIR / f'L205n{SIM_RES}TNG'
    
    inventory = {
        'snap': snap,
        'z': SNAPSHOT_REDSHIFTS[snap],
        'spherical': None,  # profiles_spherical_snap*.h5
        'bcm': None,        # profiles_bcm_snap*.h5
        'old_format': None  # snap*/profiles.h5
    }
    
    # New spherical profile (DMO + Hydro)
    spherical_file = sim_dir / f'profiles_spherical_snap{snap:03d}.h5'
    if spherical_file.exists():
        with h5py.File(spherical_file, 'r') as f:
            inventory['spherical'] = {
                'n_halos': f.attrs.get('n_halos', 0),
                'path': spherical_file,
                'profiles': list(f['profiles'].keys()) if 'profiles' in f else []
            }
    
    # New BCM profile
    bcm_file = sim_dir / f'profiles_bcm_snap{snap:03d}.h5'
    if bcm_file.exists():
        with h5py.File(bcm_file, 'r') as f:
            inventory['bcm'] = {
                'n_halos': f.attrs.get('n_halos', 0),
                'path': bcm_file,
                'profiles': list(f['profiles'].keys()) if 'profiles' in f else []
            }
    
    # Old format
    old_file = sim_dir / f'snap{snap:03d}' / 'profiles.h5'
    if old_file.exists():
        with h5py.File(old_file, 'r') as f:
            inventory['old_format'] = {
                'n_halos': f.attrs.get('n_halos', 0),
                'path': old_file,
                'keys': list(f.keys())[:5]
            }
    
    profile_inventory[snap] = inventory
    
    # Print status
    sph = '✓' if inventory['spherical'] else '✗'
    bcm = '✓' if inventory['bcm'] else '✗'
    old = '✓' if inventory['old_format'] else '✗'
    
    n_halos_sph = inventory['spherical']['n_halos'] if inventory['spherical'] else 0
    n_halos_bcm = inventory['bcm']['n_halos'] if inventory['bcm'] else 0
    
    print(f"Snap {snap:3d} (z={SNAPSHOT_REDSHIFTS[snap]:4.2f}): spherical {sph} ({n_halos_sph:4d}), BCM {bcm} ({n_halos_bcm:4d}), old {old}")

# Summary
n_spherical = sum(1 for v in profile_inventory.values() if v['spherical'])
n_bcm = sum(1 for v in profile_inventory.values() if v['bcm'])
n_old = sum(1 for v in profile_inventory.values() if v['old_format'])

print(f"\nSummary: spherical={n_spherical}/{len(SNAPSHOTS)}, BCM={n_bcm}/{len(SNAPSHOTS)}, old={n_old}/{len(SNAPSHOTS)}")

In [ ]:
# Helper functions to load profile data from different formats

def load_profiles_new_format(sim_res, snap, output_dir=OUTPUT_DIR):
    """Load profile data from new MPI format (profiles_spherical + profiles_bcm)."""
    sim_dir = output_dir / f'L205n{sim_res}TNG'
    
    data = {
        'snap': snap,
        'z': SNAPSHOT_REDSHIFTS.get(snap, None),
        'r_bins': None,
        'r_centers': None,
        'dmo_masses': None,
        'dmo_radii': None,
        'dmo_positions': None,
        'profiles': {}
    }
    
    # Load spherical profiles (DMO + Hydro)
    spherical_file = sim_dir / f'profiles_spherical_snap{snap:03d}.h5'
    if spherical_file.exists():
        with h5py.File(spherical_file, 'r') as f:
            data['r_bins'] = f['r_bins'][:]
            data['r_centers'] = f['r_centers'][:]
            data['dmo_masses'] = f['dmo_masses'][:]
            data['dmo_radii'] = f['dmo_radii'][:]
            data['dmo_positions'] = f['dmo_positions'][:]
            data['n_halos'] = f.attrs['n_halos']
            
            # Load profiles
            if 'profiles' in f:
                if 'dmo' in f['profiles']:
                    data['profiles']['dmo'] = f['profiles/dmo'][:]
                if 'hydro' in f['profiles']:
                    data['profiles']['hydro'] = f['profiles/hydro'][:]
            
            # Optional: hydro masses/positions
            if 'hydro_masses' in f:
                data['hydro_masses'] = f['hydro_masses'][:]
    
    # Load BCM profiles
    bcm_file = sim_dir / f'profiles_bcm_snap{snap:03d}.h5'
    if bcm_file.exists():
        with h5py.File(bcm_file, 'r') as f:
            if data['r_bins'] is None:
                data['r_bins'] = f['r_bins'][:]
                data['r_centers'] = f['r_centers'][:]
                data['dmo_masses'] = f['dmo_masses'][:]
                data['dmo_radii'] = f['dmo_radii'][:]
                data['dmo_positions'] = f['dmo_positions'][:]
                data['n_halos'] = f.attrs['n_halos']
            
            # Load BCM profiles
            if 'profiles' in f:
                for bcm_name in BCM_MODELS:
                    if bcm_name in f['profiles']:
                        data['profiles'][f'bcm_{bcm_name}'] = f['profiles'][bcm_name][:]
    
    return data


def load_profiles_old_format(sim_res, snap, output_dir=OUTPUT_DIR):
    """Load profile data from old format (snap*/profiles.h5)."""
    profile_path = output_dir / f'L205n{sim_res}TNG' / f'snap{snap:03d}' / 'profiles.h5'
    
    if not profile_path.exists():
        return None
    
    data = {
        'snap': snap,
        'z': SNAPSHOT_REDSHIFTS.get(snap, None),
        'profiles': {}
    }
    
    with h5py.File(profile_path, 'r') as f:
        # Radial bins
        if 'radial_bins' in f:
            data['r_bins'] = f['radial_bins'][:]
        else:
            data['r_bins'] = np.logspace(-2, 1, 31)
        
        data['r_centers'] = np.sqrt(data['r_bins'][:-1] * data['r_bins'][1:])
        
        # Halo info
        data['dmo_masses'] = f['halo_masses'][:]
        data['dmo_positions'] = f['halo_positions'][:]
        data['dmo_radii'] = f['halo_radii'][:]
        data['n_halos'] = f.attrs.get('n_halos', len(data['dmo_masses']))
        
        # Profiles
        for key in ['dmo_profiles', 'hydro_profiles', 'replace_profiles']:
            if key in f:
                name = key.replace('_profiles', '')
                data['profiles'][name] = f[key][:]
        
        # BCM profiles
        for bcm in ['arico20', 'schneider19', 'schneider25']:
            key = f'bcm_{bcm}_profiles'
            if key in f:
                data['profiles'][f'bcm_{bcm}'] = f[key][:]
    
    return data


def load_profiles(sim_res, snap, output_dir=OUTPUT_DIR):
    """Load profiles - try new format first, fall back to old format."""
    data = load_profiles_new_format(sim_res, snap, output_dir)
    if data['r_bins'] is not None:
        data['format'] = 'new'
        return data
    
    data = load_profiles_old_format(sim_res, snap, output_dir)
    if data is not None:
        data['format'] = 'old'
        return data
    
    return None

print("Helper functions defined: load_profiles(), load_profiles_new_format(), load_profiles_old_format()")

In [ ]:
# Load profile data for a specific snapshot
SNAP = 99  # Change as needed

data = load_profiles(SIM_RES, SNAP)

if data is None:
    print(f"No profile data found for snapshot {SNAP}!")
else:
    print(f"Profile Data Summary - Snapshot {SNAP} (z={data['z']:.2f})")
    print("=" * 70)
    print(f"Format: {data.get('format', 'unknown')}")
    print(f"Number of halos: {data['n_halos']}")
    print(f"Mass range: log10(M) = {np.log10(data['dmo_masses'].min()):.2f} - {np.log10(data['dmo_masses'].max()):.2f}")
    print(f"Radial bins: {len(data['r_centers'])} bins from {data['r_centers'][0]:.3f} to {data['r_centers'][-1]:.2f} r/R200")
    
    print(f"\nAvailable profiles:")
    for name, prof in data['profiles'].items():
        n_complete = np.sum(np.all(prof > 0, axis=1))
        n_partial = np.sum(np.any(prof > 0, axis=1)) - n_complete
        print(f"  {name}: shape {prof.shape}, {n_complete} complete, {n_partial} partial")
    
    print(f"\nHalos by mass bin:")
    for i, (mass_lo, mass_hi, label) in enumerate(zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS)):
        n = np.sum((data['dmo_masses'] >= mass_lo) & (data['dmo_masses'] < mass_hi))
        print(f"  log10(M) = {label}: {n} halos")

In [ ]:
# Profile completeness analysis
if data is not None:
    profiles = data['profiles']
    r_centers = data['r_centers']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Completeness by radial bin
    ax = axes[0]
    for name, p in profiles.items():
        completeness = np.sum(p > 0, axis=0) / len(p) * 100
        ax.semilogx(r_centers, completeness, '-o', label=name, markersize=4, alpha=0.7)
    
    ax.set_xlabel('r / R200')
    ax.set_ylabel('Completeness [%]')
    ax.set_title(f'Profile Completeness by Radial Bin (snap {SNAP}, z={data["z"]:.2f})')
    ax.legend(fontsize=8)
    ax.axhline(100, color='gray', linestyle='--', alpha=0.5)
    ax.set_ylim(0, 105)
    ax.grid(True, alpha=0.3)
    
    # Distribution of non-zero bins per halo
    ax = axes[1]
    for name, p in profiles.items():
        n_nonzero_bins = np.sum(p > 0, axis=1)
        ax.hist(n_nonzero_bins, bins=np.arange(0, len(r_centers)+2)-0.5, 
                alpha=0.5, label=name, density=True)
    
    ax.set_xlabel('Number of non-zero radial bins')
    ax.set_ylabel('Density')
    ax.set_title('Profile Completeness per Halo')
    ax.legend(fontsize=8)
    ax.axvline(len(r_centers), color='red', linestyle='--', label='All bins')
    
    plt.tight_layout()
    plt.show()
else:
    print("No data to plot")

In [ ]:
# Stacked profiles by mass bin
def stack_profiles(profile_array, masses, mass_lo, mass_hi, method='median'):
    """Stack profiles for halos in a mass range.
    
    Args:
        profile_array: (n_halos, n_bins) array
        masses: halo masses
        mass_lo, mass_hi: mass range
        method: 'median' or 'mean'
    
    Returns:
        stacked profile, lower 16th percentile, upper 84th percentile, count
    """
    mask = (masses >= mass_lo) & (masses < mass_hi)
    if np.sum(mask) == 0:
        return None, None, None, 0
    
    selected = profile_array[mask]
    
    # Use halos with complete profiles for stacking
    complete_mask = np.all(selected > 0, axis=1)
    n_complete = np.sum(complete_mask)
    
    if n_complete < 3:
        # Fall back to partial profiles
        median = np.zeros(selected.shape[1])
        lower = np.zeros(selected.shape[1])
        upper = np.zeros(selected.shape[1])
        for i in range(selected.shape[1]):
            vals = selected[:, i]
            vals = vals[vals > 0]
            if len(vals) > 0:
                if method == 'median':
                    median[i] = np.median(vals)
                else:
                    median[i] = np.mean(vals)
                lower[i] = np.percentile(vals, 16)
                upper[i] = np.percentile(vals, 84)
        return median, lower, upper, np.sum(mask)
    
    complete_profiles = selected[complete_mask]
    if method == 'median':
        stacked = np.median(complete_profiles, axis=0)
    else:
        stacked = np.mean(complete_profiles, axis=0)
    
    lower = np.percentile(complete_profiles, 16, axis=0)
    upper = np.percentile(complete_profiles, 84, axis=0)
    
    return stacked, lower, upper, n_complete


if data is not None:
    profiles = data['profiles']
    masses = data['dmo_masses']
    r_centers = data['r_centers']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for i, (mass_lo, mass_hi, label) in enumerate(zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS)):
        ax = axes[i]
        
        n_in_bin = np.sum((masses >= mass_lo) & (masses < mass_hi))
        
        for name, p in profiles.items():
            stacked, lower, upper, n = stack_profiles(p, masses, mass_lo, mass_hi)
            if stacked is not None and np.any(stacked > 0):
                ax.loglog(r_centers, stacked, '-', label=f'{name} ({n})', alpha=0.8)
                if lower is not None:
                    ax.fill_between(r_centers, lower, upper, alpha=0.15)
        
        ax.set_xlabel('r / R200')
        ax.set_ylabel('Density [Msun/Mpc³]')
        ax.set_title(f'log₁₀(M) = {label} ({n_in_bin} halos)')
        ax.legend(fontsize=7, loc='upper right')
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Stacked Density Profiles - Snap {SNAP} (z={data["z"]:.2f})', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No data to plot")

In [ ]:
# Ratio profiles (Hydro/DMO, BCM/DMO)
if data is not None and 'dmo' in data['profiles'] and len(data['profiles']) > 1:
    profiles = data['profiles']
    masses = data['dmo_masses']
    r_centers = data['r_centers']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for i, (mass_lo, mass_hi, label) in enumerate(zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS)):
        ax = axes[i]
        
        dmo_stacked, _, _, n_dmo = stack_profiles(profiles['dmo'], masses, mass_lo, mass_hi)
        
        if dmo_stacked is not None and np.any(dmo_stacked > 0):
            for name, p in profiles.items():
                if name == 'dmo':
                    continue
                
                stacked, _, _, n = stack_profiles(p, masses, mass_lo, mass_hi)
                if stacked is not None and np.any(stacked > 0):
                    ratio = np.ones_like(stacked)
                    mask = dmo_stacked > 0
                    ratio[mask] = stacked[mask] / dmo_stacked[mask]
                    ax.semilogx(r_centers, ratio, '-', label=f'{name} ({n})', alpha=0.8)
        
        ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5)
        ax.set_xlabel('r / R200')
        ax.set_ylabel('Ratio to DMO')
        ax.set_title(f'log₁₀(M) = {label}')
        ax.legend(fontsize=7)
        ax.set_ylim(0.3, 1.7)
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Profile Ratios to DMO - Snap {SNAP} (z={data["z"]:.2f})', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("Need DMO profiles to compute ratios")

In [ ]:
# Individual halo examples by mass bin
if data is not None and 'dmo' in data['profiles']:
    profiles = data['profiles']
    masses = data['dmo_masses']
    r_centers = data['r_centers']
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    np.random.seed(42)
    
    for i, (mass_lo, mass_hi, label) in enumerate(zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS)):
        ax = axes[i]
        
        # Find halos in this mass bin with complete DMO profiles
        mass_mask = (masses >= mass_lo) & (masses < mass_hi)
        dmo_complete = np.all(profiles['dmo'] > 0, axis=1)
        valid_idx = np.where(mass_mask & dmo_complete)[0]
        
        if len(valid_idx) == 0:
            ax.text(0.5, 0.5, f'No complete profiles\nin mass bin {label}', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'log₁₀(M) = {label}')
            continue
        
        # Pick one random halo
        idx = np.random.choice(valid_idx)
        
        for name, p in profiles.items():
            prof = p[idx]
            if np.any(prof > 0):
                ax.loglog(r_centers, prof, '-', label=name, alpha=0.8)
        
        ax.set_xlabel('r / R200')
        ax.set_ylabel('Density [Msun/Mpc³]')
        ax.set_title(f'Halo {idx}: log₁₀(M) = {np.log10(masses[idx]):.2f}')
        ax.legend(fontsize=7)
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Individual Halo Profiles - Snap {SNAP} (z={data["z"]:.2f})', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No DMO profiles available")

In [ ]:
# Summary statistics
if data is not None:
    profiles = data['profiles']
    masses = data['dmo_masses']
    r_centers = data['r_centers']
    
    print("=" * 70)
    print(f"PROFILE SUMMARY - Snapshot {SNAP} (z={data['z']:.2f})")
    print("=" * 70)
    
    print(f"\nTotal halos: {len(masses)}")
    print(f"Mass range: log₁₀(M) = {np.log10(masses.min()):.2f} - {np.log10(masses.max()):.2f}")
    print(f"Radial bins: {len(r_centers)} from {r_centers[0]:.3f} to {r_centers[-1]:.2f} r/R200")
    
    print("\nProfile completeness:")
    for name, p in profiles.items():
        n_complete = np.sum(np.all(p > 0, axis=1))
        n_partial = np.sum(np.any(p > 0, axis=1)) - n_complete
        n_empty = np.sum(~np.any(p > 0, axis=1))
        print(f"  {name}: {n_complete} complete, {n_partial} partial, {n_empty} empty")
    
    print("\nHalos by mass bin:")
    for mass_lo, mass_hi, label in zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS):
        n = np.sum((masses >= mass_lo) & (masses < mass_hi))
        print(f"  log₁₀(M) = {label}: {n}")
else:
    print("No data loaded")

In [ ]:
# =============================================================================
# MULTI-SNAPSHOT / REDSHIFT ANALYSIS
# =============================================================================

# Load all available snapshots
all_data = {}
for snap in SNAPSHOTS:
    d = load_profiles(SIM_RES, snap)
    if d is not None and d['r_bins'] is not None:
        all_data[snap] = d

print(f"Loaded {len(all_data)} snapshots: {sorted(all_data.keys())}")
print(f"Redshift range: z={SNAPSHOT_REDSHIFTS[max(all_data.keys())]:.2f} to z={SNAPSHOT_REDSHIFTS[min(all_data.keys())]:.2f}")

In [ ]:
# Profile ratios vs redshift for different mass bins
if len(all_data) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    # Colors for different models
    model_colors = {
        'hydro': 'tab:blue',
        'bcm_Arico20': 'tab:orange',
        'bcm_Schneider19': 'tab:green',
        'bcm_Schneider25': 'tab:red'
    }
    
    for i, (mass_lo, mass_hi, label) in enumerate(zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS)):
        ax = axes[i]
        
        # Collect ratio at r = 0.5 R200 and r = 1.0 R200 for each snapshot
        for profile_name in ['hydro', 'bcm_Arico20', 'bcm_Schneider19', 'bcm_Schneider25']:
            redshifts = []
            ratios_half = []  # r ~ 0.5 R200
            ratios_one = []   # r ~ 1.0 R200
            
            for snap, d in sorted(all_data.items(), key=lambda x: x[0]):
                if 'dmo' not in d['profiles'] or profile_name not in d['profiles']:
                    continue
                
                masses = d['dmo_masses']
                r_centers = d['r_centers']
                
                # Find radial bin closest to 0.5 and 1.0 R200
                idx_half = np.argmin(np.abs(r_centers - 0.5))
                idx_one = np.argmin(np.abs(r_centers - 1.0))
                
                dmo_stacked, _, _, n_dmo = stack_profiles(d['profiles']['dmo'], masses, mass_lo, mass_hi)
                other_stacked, _, _, n_other = stack_profiles(d['profiles'][profile_name], masses, mass_lo, mass_hi)
                
                if n_dmo > 0 and n_other > 0 and dmo_stacked[idx_half] > 0 and dmo_stacked[idx_one] > 0:
                    z = SNAPSHOT_REDSHIFTS[snap]
                    redshifts.append(z)
                    ratios_half.append(other_stacked[idx_half] / dmo_stacked[idx_half])
                    ratios_one.append(other_stacked[idx_one] / dmo_stacked[idx_one])
            
            if len(redshifts) > 0:
                color = model_colors.get(profile_name, 'gray')
                ax.plot(redshifts, ratios_half, 'o-', label=f'{profile_name} (0.5 R200)', 
                       color=color, alpha=0.8)
                ax.plot(redshifts, ratios_one, 's--', label=f'{profile_name} (1.0 R200)', 
                       color=color, alpha=0.5)
        
        ax.axhline(1.0, color='gray', linestyle=':', alpha=0.5)
        ax.set_xlabel('Redshift z')
        ax.set_ylabel('Density Ratio to DMO')
        ax.set_title(f'log₁₀(M) = {label}')
        ax.legend(fontsize=6, ncol=2, loc='upper right')
        ax.set_ylim(0.3, 1.5)
        ax.invert_xaxis()  # High z on left
        ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Profile Ratios vs Redshift (L205n{SIM_RES}TNG)', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No data loaded")

In [ ]:
# Profile evolution with redshift - select one mass bin
SELECTED_MASS_BIN = 1  # 0=12.5-13.0, 1=13.0-13.5, 2=13.5-14.0, 3=>14.0

mass_lo = MASS_BIN_EDGES[SELECTED_MASS_BIN]
mass_hi = MASS_BIN_EDGES[SELECTED_MASS_BIN + 1]
mass_label = MASS_BIN_LABELS[SELECTED_MASS_BIN]

if len(all_data) > 0:
    # Pick a subset of snapshots to show evolution
    snaps_to_plot = [s for s in sorted(all_data.keys()) if s in [99, 85, 67, 49, 29]]
    if len(snaps_to_plot) == 0:
        snaps_to_plot = sorted(all_data.keys())[::len(all_data)//5 + 1]  # Evenly spaced
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(snaps_to_plot)))
    
    # Left: DMO profiles at different redshifts
    ax = axes[0]
    for snap, color in zip(snaps_to_plot, colors):
        d = all_data[snap]
        if 'dmo' not in d['profiles']:
            continue
        
        stacked, _, _, n = stack_profiles(d['profiles']['dmo'], d['dmo_masses'], mass_lo, mass_hi)
        if stacked is not None and np.any(stacked > 0):
            z = SNAPSHOT_REDSHIFTS[snap]
            ax.loglog(d['r_centers'], stacked, '-', color=color, label=f'z={z:.2f} ({n})', alpha=0.8)
    
    ax.set_xlabel('r / R200')
    ax.set_ylabel('Density [Msun/Mpc³]')
    ax.set_title(f'DMO Profiles - log₁₀(M) = {mass_label}')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Right: Hydro/DMO ratio at different redshifts
    ax = axes[1]
    for snap, color in zip(snaps_to_plot, colors):
        d = all_data[snap]
        if 'dmo' not in d['profiles'] or 'hydro' not in d['profiles']:
            continue
        
        dmo_stacked, _, _, n_dmo = stack_profiles(d['profiles']['dmo'], d['dmo_masses'], mass_lo, mass_hi)
        hydro_stacked, _, _, n_hydro = stack_profiles(d['profiles']['hydro'], d['dmo_masses'], mass_lo, mass_hi)
        
        if dmo_stacked is not None and hydro_stacked is not None:
            ratio = np.ones_like(dmo_stacked)
            mask = dmo_stacked > 0
            ratio[mask] = hydro_stacked[mask] / dmo_stacked[mask]
            
            z = SNAPSHOT_REDSHIFTS[snap]
            ax.semilogx(d['r_centers'], ratio, '-', color=color, label=f'z={z:.2f}', alpha=0.8)
    
    ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5)
    ax.set_xlabel('r / R200')
    ax.set_ylabel('Hydro / DMO')
    ax.set_title(f'Hydro/DMO Ratio - log₁₀(M) = {mass_label}')
    ax.legend(fontsize=8)
    ax.set_ylim(0.3, 1.5)
    ax.grid(True, alpha=0.3)
    
    plt.suptitle(f'Profile Evolution with Redshift (L205n{SIM_RES}TNG)', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No data loaded")

In [ ]:
# Summary table across all snapshots
if len(all_data) > 0:
    print("=" * 90)
    print(f"MULTI-SNAPSHOT SUMMARY (L205n{SIM_RES}TNG)")
    print("=" * 90)
    
    print(f"\n{'Snap':>5} {'z':>6} {'N_halos':>8} {'N_dmo':>8} {'N_hydro':>8} {'N_bcm':>8}")
    print("-" * 90)
    
    for snap in sorted(all_data.keys(), reverse=True):
        d = all_data[snap]
        n_halos = d['n_halos']
        n_dmo = np.sum(np.any(d['profiles'].get('dmo', np.zeros((1,1))) > 0, axis=1)) if 'dmo' in d['profiles'] else 0
        n_hydro = np.sum(np.any(d['profiles'].get('hydro', np.zeros((1,1))) > 0, axis=1)) if 'hydro' in d['profiles'] else 0
        
        # Count BCM profiles
        bcm_count = 0
        for key in d['profiles']:
            if key.startswith('bcm_'):
                bcm_count = np.sum(np.any(d['profiles'][key] > 0, axis=1))
                break  # Just count one BCM model
        
        z = SNAPSHOT_REDSHIFTS[snap]
        print(f"{snap:>5} {z:>6.2f} {n_halos:>8} {n_dmo:>8} {n_hydro:>8} {bcm_count:>8}")
    
    print("\nProfiles available by mass bin (using latest snapshot):")
    latest_snap = max(all_data.keys())
    d = all_data[latest_snap]
    
    print(f"\n{'Mass bin':>15} {'Total':>8} {'DMO':>8} {'Hydro':>8}")
    print("-" * 50)
    for mass_lo, mass_hi, label in zip(MASS_BIN_EDGES[:-1], MASS_BIN_EDGES[1:], MASS_BIN_LABELS):
        mask = (d['dmo_masses'] >= mass_lo) & (d['dmo_masses'] < mass_hi)
        n_total = np.sum(mask)
        
        n_dmo = 0
        n_hydro = 0
        if 'dmo' in d['profiles']:
            n_dmo = np.sum(mask & np.all(d['profiles']['dmo'] > 0, axis=1))
        if 'hydro' in d['profiles']:
            n_hydro = np.sum(mask & np.all(d['profiles']['hydro'] > 0, axis=1))
        
        print(f"{label:>15} {n_total:>8} {n_dmo:>8} {n_hydro:>8}")
else:
    print("No data loaded")